In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE, Isomap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Normalizer
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
import random
import copy

In [ ]:
%cd /content/drive/MyDrive/2022/BigData
file_root = './fifa_dataset_with_value.csv'
test_root='./'
df_old =  pd.read_csv(file_root, encoding='cp949')

/content/drive/MyDrive/2022/BigData


In [ ]:
df = pd.read_csv(file_root, encoding='cp949')
#date 형식 변환
#df['date'] = [i.replace('-','') for i in df['date']]
#all_list : tournament,home_team, away_team
all_list = ((df['tournament'].unique()).tolist() + (df['home_team'].unique()).tolist() + (df['away_team'].unique()).tolist())

uni_len = '***'.join(all_list)
nat_enc = tf.keras.preprocessing.text.Tokenizer(num_words=len(all_list),
    filters='',
    lower=False,
    split = '***')
nat_enc.fit_on_texts([uni_len])
nat_dic = nat_enc.word_index

In [ ]:
#값을 enumerate로 변환
for step, val in enumerate(df['tournament']):
    df['tournament'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['home_team']):
    df['home_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['away_team']):
    df['away_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['neutral']):
    if df['neutral'].iloc[step] == False:
        df['neutral'].iloc[step] = 0
    if df['neutral'].iloc[step] == True:
        df['neutral'].iloc[step] = 1

In [ ]:
df.head()

,home_team,away_team,home_score,away_score,tournament,neutral,home_team_total_points,away_team_total_points,home_B1_point,home_B2_point,...,away_B2_value,away_B3_value,away_B4_value,away_M1_value,away_M2_value,away_M3_value,away_M4_value,away_T1_value,away_T2_value,away_K1_value
0,1,10,2,1,91,1,603.0,656.0,73,72,...,1100000,1300000,775000,7000000,5500000,2900000,3300000,13000000,3300000,10000000
1,2,58,1,0,91,1,714.0,529.0,73,72,...,100000,130000,90000,2400000,1700000,3800000,550000,2400000,3800000,525000
2,3,1,1,1,91,0,579.0,603.0,72,72,...,2000000,1300000,475000,1300000,3900000,1700000,1500000,6500000,3900000,1500000
3,4,25,0,3,92,1,500.0,680.0,74,74,...,2500000,2800000,2300000,7000000,12500000,4000000,3600000,44000000,15500000,10500000
4,3,10,1,3,91,0,579.0,656.0,72,72,...,1100000,1300000,775000,7000000,5500000,2900000,3300000,13000000,3300000,10000000


In [ ]:
#df_new
df_new = df
#len
temp_tor = []
for i in df_old['tournament']:
    temp_tor.append(len(i))
temp_away = []
for i in df_old['away_team']:
    temp_away.append(len(i))
temp_home = []
for i in df_old['home_team']:
    temp_home.append(len(i))
df_new['tor_len'] = temp_tor
df_new['home_len'] = temp_home
df_new['away_len'] = temp_away

# 결측값 있는 행 제거
df_new = df_new.dropna()

df_label1 = df_new['home_score']
df_label2 = df_new['away_score']

In [ ]:
#df_new, df_label 분리 : 모델의 인풋(df_new)과 아웃풋(df_label, home_score 과 away score)
df_new = df_new.drop(['home_score', 'away_score'], axis = 1)
#df_label
df_label = pd.concat([df_label1, df_label2],axis = 1)
df_label.columns = ['home_score', 'away_score']

**df_label** : 왜 인덱스가 7948까지 있을까? 표시는 6547 rows라고 되긴 함

In [ ]:
# concat이 단순하게 합치는 연산이라 index 갱신이 안된듯 

In [ ]:
df_label

,home_score,away_score
0,2,1
1,1,0
2,1,1
3,0,3
4,1,3
...,...,...
2543,0,0
2544,0,5
2545,1,1
2546,0,0


In [ ]:
x_train = df_new.to_numpy()
df_train = pd.DataFrame(data=x_train, columns=df_new.columns)

**df_train**

In [ ]:
df_train

,home_team,away_team,tournament,neutral,home_team_total_points,away_team_total_points,home_B1_point,home_B2_point,home_B3_point,home_B4_point,...,away_M1_value,away_M2_value,away_M3_value,away_M4_value,away_T1_value,away_T2_value,away_K1_value,tor_len,home_len,away_len
0,1,10,91,1,603.0,656.0,73,72,70,70,...,7000000,5500000,2900000,3300000,13000000,3300000,10000000,28,4,7
1,2,58,91,1,714.0,529.0,73,72,71,71,...,2400000,1700000,3800000,550000,2400000,3800000,525000,28,5,5
2,3,1,91,0,579.0,603.0,72,72,72,70,...,1300000,3900000,1700000,1500000,6500000,3900000,1500000,28,12,4
3,4,25,92,1,500.0,680.0,74,74,73,72,...,7000000,12500000,4000000,3600000,44000000,15500000,10500000,8,6,6
4,3,10,91,0,579.0,656.0,72,72,72,70,...,7000000,5500000,2900000,3300000,13000000,3300000,10000000,28,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,85,16,92,0,1243.8,1335.36,67,66,61,58,...,1400000,975000,1800000,875000,2600000,1300000,725000,8,10,8
2544,82,27,92,0,961.23,1434.68,61,58,57,47,...,4800000,5000000,1500000,3500000,17000000,4700000,825000,8,7,7
2545,4,34,92,0,1473.04,1405.6,77,77,75,75,...,7500000,3000000,2400000,2300000,4600000,3000000,18000000,8,6,7
2546,55,6,92,0,1425.59,1509.61,77,75,74,73,...,10500000,9500000,14000000,2800000,11000000,11500000,1600000,8,8,5


In [ ]:
#x_train = df_new.to_numpy()
Norm = Normalizer() # 각 변수의 값을 원점으로부터 1만큼 떨어져 있는 값으로 변환 
x_temp = Norm.fit_transform(x_train)
df_temp = pd.DataFrame(data=x_temp, columns=df_new.columns)

**df_temp** : df_train은 x_train을 사용, df_temp는 x_train을 normalize한 값인 x_temp 사용

In [ ]:
df_temp

,home_team,away_team,tournament,neutral,home_team_total_points,away_team_total_points,home_B1_point,home_B2_point,home_B3_point,home_B4_point,...,away_M1_value,away_M2_value,away_M3_value,away_M4_value,away_T1_value,away_T2_value,away_K1_value,tor_len,home_len,away_len
0,4.574773e-08,4.574773e-07,4.163043e-06,4.574773e-08,0.000028,0.000030,3.339584e-06,3.293837e-06,3.202341e-06,3.202341e-06,...,0.320234,0.251613,0.132668,0.150968,0.594720,0.150968,0.457477,1.280936e-06,1.829909e-07,3.202341e-07
1,1.023027e-07,2.966779e-06,4.654773e-06,5.115136e-08,0.000037,0.000027,3.734049e-06,3.682898e-06,3.631746e-06,3.631746e-06,...,0.122763,0.086957,0.194375,0.028133,0.122763,0.194375,0.026854,1.432238e-06,2.557568e-07,2.557568e-07
2,2.160252e-07,7.200840e-08,6.552764e-06,0.000000e+00,0.000042,0.000043,5.184605e-06,5.184605e-06,5.184605e-06,5.040588e-06,...,0.093611,0.280833,0.122414,0.108013,0.468055,0.280833,0.108013,2.016235e-06,8.641008e-07,2.880336e-07
3,7.678830e-08,4.799269e-07,1.766131e-06,1.919708e-08,0.000010,0.000013,1.420584e-06,1.420584e-06,1.401387e-06,1.382189e-06,...,0.134380,0.239963,0.076788,0.069109,0.844671,0.297555,0.201569,1.535766e-07,1.151825e-07,1.151825e-07
4,1.353301e-07,4.511002e-07,4.105012e-06,0.000000e+00,0.000026,0.000030,3.247922e-06,3.247922e-06,3.247922e-06,3.157702e-06,...,0.315770,0.248105,0.130819,0.148863,0.586430,0.148863,0.451100,1.263081e-06,5.413203e-07,3.157702e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,9.965277e-06,1.875817e-06,1.078595e-05,0.000000e+00,0.000146,0.000157,7.854983e-06,7.737744e-06,7.151551e-06,6.799836e-06,...,0.164134,0.114308,0.211029,0.102584,0.304820,0.152410,0.084998,9.379084e-07,1.172385e-06,9.379084e-07
2544,4.069958e-06,1.340108e-06,4.566295e-06,0.000000e+00,0.000048,0.000071,3.027652e-06,2.878751e-06,2.829117e-06,2.332781e-06,...,0.238241,0.248168,0.074450,0.173718,0.843772,0.233278,0.040948,3.970691e-07,3.474355e-07,3.474355e-07
2545,2.431057e-08,2.066398e-07,5.591431e-07,0.000000e+00,0.000009,0.000009,4.679785e-07,4.679785e-07,4.558232e-07,4.558232e-07,...,0.045582,0.018233,0.014586,0.013979,0.027957,0.018233,0.109398,4.862114e-08,3.646586e-08,4.254350e-08
2546,6.554392e-07,7.150245e-08,1.096371e-06,0.000000e+00,0.000017,0.000018,9.176148e-07,8.937807e-07,8.818636e-07,8.699465e-07,...,0.125129,0.113212,0.166839,0.033368,0.131088,0.137046,0.019067,9.533661e-08,9.533661e-08,5.958538e-08


In [ ]:
df_corr = df_temp.corr() # 각 열간의 상관계수 반환 

df_label과 df_temp(normalize된 값)

In [ ]:
y_es = df_label.to_numpy()
x_es = df_temp.to_numpy()
x_t, x_val , y_t, y_val = train_test_split(x_es, y_es, random_state=28)

In [ ]:
params = {
    "estimator__colsample_bylevel": uniform(0.7, 0.3),
    "estimator__gamma": uniform(0, 0.5),
    "estimator__learning_rate": uniform(0.003, 0.3),
    "estimator__max_depth": randint(2, 6),
    "estimator__n_estimators": randint(100, 500), 
    "estimator__subsample": uniform(0.6, 0.4)
}

In [ ]:
#Multioutput regression are regression problems that involve predicting two or more numerical values given an input example.
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', eval_metric=['mae'], random_state = 1, use_label_encoder=False))

In [ ]:
#search : 하이퍼 파라미터 튜닝
search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state =1, n_iter=5, cv=3,
                            verbose=0, n_jobs=1, return_train_score=True) 
search.fit(x_t, y_t) 

best_param = search.best_params_
print('-')
print(search.best_score_)
print(best_param)
print('-')

-
0.10661130281389992
{'estimator__colsample_bylevel': 0.9147911547905092, 'estimator__gamma': 0.40137875196868245, 'estimator__learning_rate': 0.030840242592221373, 'estimator__max_depth': 3, 'estimator__n_estimators': 101, 'estimator__subsample': 0.9460081007915934}
-


In [ ]:
#RandomizedSearchCV로 찾은 best param을 모델에 적용
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror',eval_metric=['mae'], random_state = 1, 
                              use_label_encoder=False, **best_param)).fit(x_t, y_t) 

In [ ]:
#MAE : MSE와 유사한 loss function
#이 코드에서 loss function이 적용되는 부분이 어디지? 못 찾았음
print('MAE: ',mean_absolute_error(y_val, xgb_model.predict(x_val)))

MAE:  0.8758363738492294


In [ ]:
#K-fold 교차 검증:
#전체 훈련 데이터를 K개의 그룹으로 묶고, 그중 한 그룹을 검증셋으로 사용합니다. 
#이때 각각의 모든 데이터 그룹이 적어도 한 번씩은 검증셋으로 사용되어야 합니다. 
#K-fold 교차 검증을 사용함으로써 검정 데이터에 모델이 과적합 되는 것을 막을 수 있습니다. 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(xgb_model, x_val, y_val, scoring='neg_mean_absolute_error', cv=folds, verbose=0)
print('KFold_Neg_MAE: ',np.mean(scores))

KFold_Neg_MAE:  -0.9084265330910875


======================여기까지가 모델 트레이닝========================

# **predict_score 변경 예정**

In [ ]:
file_WC_team = './input_test.csv'
df_teams = pd.read_csv(file_WC_team, encoding='cp949')

In [ ]:
#inp_list: 사용자가 입력한 home_team, away team
def predict_score(inp_list, autofill, Norm, seed, model):
    # ind_list = []
    # for i in inp_list:
    #     ind_list.append(nat_dic.get(i))
    # seed[:,1] = ind_list[0]
    # seed[:,2] = ind_list[1]
    x_test = []
    qatar_flag = False
    home_away_chagned_flag = False

    home_team = inp_list[0]
    away_team = inp_list[1]

    # Netural == False인 유일한 상황 체크
    if (home_team == 'Qatar' or away_team == 'Qatar'):
        qatar_flag = True
        if(away_team == 'Qatar'):
            home_away_chagned_flag = True
            home_team, away_team = away_team, home_team

    home_data = df_teams.loc[df_teams['team'] == home_team]
    away_data = df_teams.loc[df_teams['team'] == away_team]


    # Home_team norm 값
    x_test.append(nat_dic.get(home_team))
    # Away_team norm 값
    x_test.append(nat_dic.get(away_team))

    # Tournament norm 값
    x_test.append(nat_dic.get('FIFA World Cup'))

    # Neutral 값
    if (qatar_flag):
        x_test.append(0)
    else:
        x_test.append(1)

    # Rank 값
    x_test.append(float(home_data.iloc[0]['total_points']))
    x_test.append(float(away_data.iloc[0]['total_points']))

    # 선수 능력치 값
    feature_num = 2                   # TODO: 최종 Feature 정한 후 값 수정 필요
    for i in range(0, feature_num):
      x_test += home_data.iloc[0][2+(i*11):13+(i*11)].to_list()
      x_test += away_data.iloc[0][2+(i*11):13+(i*11)].to_list()

    # Tournament 이름 len 값
    x_test.append(len('FIFA World Cup'))
    # Home_team 이름 len 값
    x_test.append(len(home_team))
    # Away_team 이름 len 값
    x_test.append(len(away_team))


    # Norm
    x_test_pd = pd.DataFrame(data=np.array(x_test).reshape(-1, len(x_test)), columns=df_new.columns)
    x_test_norm = Norm.fit_transform(x_test_pd)
    pred = model.predict(x_test_norm)

    output = np.ravel(pred).tolist()

    if (home_away_chagned_flag):
        output[0], output[1] = output[1], output[0]

    return np.ravel(pred).tolist()

# **mixed** : seed = np.delete를 주석처리하면 해결됨. 기존 코드 체크하기

In [ ]:
def create_params():
  seed = x_train[random.randint(0,x_train.shape[0])].reshape(1,-1)
  params = {
    'autofill': True,
    'Norm': Norm,
    'seed': seed,
    'model': xgb_model
  }
  return params

In [ ]:

#예선 팀 조 편성 결과 입력
Group_A= ["Netherlands","Senegal","Ecuador","Qatar"]
Group_B= ["England","United States","Iran","Wales"] # United States 가 없음 ;; 
Group_C= ["Argentina","Poland","Mexico", "Saudi Arabia"]
Group_D= ["France","Australia","Tunisia","Denmark"]
Group_E= ["Japan","Spain","Germany","Costa Rica"]
Group_F= ["Morocco","Croatia","Belgium","Canada"]
Group_G= ["Brazil","Switzerland","Cameroon","Serbia"]
Group_H= ["Portugal","South Korea","Uruguay","Ghana"] # South Korea 가 없음 ;; 
Groups={"Group A":Group_A,"Group B":Group_B,"Group C":Group_C,"Group D":Group_D,"Group E":Group_E,"Group F":Group_F,"Group G":Group_G,"Group H":Group_H}


In [ ]:
def create_input(home_team, away_team):
  #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
  ind_list = [home_team, away_team]
  return ind_list#input_vector

In [ ]:
def select_winning_team(ans, qual): #probability_array
    pred_ans = copy.deepcopy(ans)
    #예선 : 소수점 x, 정수로 승패 판결
    if qual == True :
      #정수
      pred_ans[0] = round(pred_ans[0])
      pred_ans[1] = round(pred_ans[1])
      
    #본선
    else :
      #소수점 1자리
      pred_ans[0] = round(pred_ans[0], 1)
      pred_ans[1] = round(pred_ans[1], 1)
    
    if (pred_ans[0]>pred_ans[1]):
          out=0 # home이 우승
    elif (pred_ans[0]<pred_ans[1]):
          out=1 # away가 우승
    elif (pred_ans[0]==pred_ans[1]):
          out=2 #비김

    return out, pred_ans

In [ ]:

##Group stage Matches
print("========Qualifying Games=========")
Group_standings={}
for grp_name in list(Groups.keys()):
    print(f"[{grp_name} Matches]")
    #해당 그룹의 팀들
    probable_countries=Groups[grp_name]

    team_wins_dct={}
    goal_scored_dct={}
    goal_against_dct={}
    win_dct={}
    draw_dct={}
    lost_dct={}
    
    for i in range(len(probable_countries)):
        #team i: team1, team j : team2
        j=i+1
        team_1=probable_countries[i]

        team_wins=0
        
        while j<len((probable_countries)):
            team_2=probable_countries[j]
            team_lst=[team_1,team_2]

            #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
            input=create_input(team_1, team_2)#np.array([[year,stadium_num,team_1_num,team_2_num]])
            params = create_params()
            #ans : 소수점 ex 1.3 : 1.5
            ans = predict_score(input, **params)

            #qual : 예선이면 true, 본선이면 false
            qual = True
           
            win, prob_lst=select_winning_team(ans, qual)
            print("applied_score : ", prob_lst,", predicted_score: ", ans)
            
            #본인 팀 스코어
            goal_scored_dct[team_1] = goal_scored_dct.get(team_1,0)+prob_lst[0]
            goal_scored_dct[team_2] = goal_scored_dct.get(team_2,0)+prob_lst[1]
            #상대 팀 스코어
            goal_against_dct[team_1] = goal_against_dct.get(team_1,0)+prob_lst[1]
            goal_against_dct[team_2] = goal_against_dct.get(team_2,0)+prob_lst[0]

            try:
                print(f" {team_1} vs {team_2} \n  {team_lst[win]} wins \n")
                #team1 우승
                if (win)==0:
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+2
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)
                    #team1 1승 추가
                    win_dct[team_1] = win_dct.get(team_1,0)+1
                    win_dct[team_2] = win_dct.get(team_2,0)
                    #team2 1패 추가
                    lost_dct[team_2] = lost_dct.get(team_2,0)+1
                    lost_dct[team_1] = lost_dct.get(team_1,0)
                    #비긴 팀 없음
                    draw_dct[team_2] = draw_dct.get(team_2,0)
                    draw_dct[team_1] = draw_dct.get(team_1,0)

                elif (win)==1:
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+2
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)
                    #team2 1승 추가
                    win_dct[team_2] = win_dct.get(team_2,0)+1
                    win_dct[team_1] = win_dct.get(team_1,0)
                    #team1 1패 추가
                    lost_dct[team_1] = lost_dct.get(team_1,0)+1
                    lost_dct[team_2] = lost_dct.get(team_2,0)
                    #비긴 팀 없음
                    draw_dct[team_1] = draw_dct.get(team_1,0)
                    draw_dct[team_2] = draw_dct.get(team_2,0)

            except IndexError:
              #비긴 경우
              print(f"{team_1} vs {team_2} \n  Match Draw \n") 
              team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+1
              team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+1
              
              #비김
              draw_dct[team_1] = draw_dct.get(team_1,0)+1
              draw_dct[team_2] = draw_dct.get(team_2,0)+1
              #승패 없음
              win_dct[team_1] = win_dct.get(team_1,0)
              lost_dct[team_1] = lost_dct.get(team_1,0)
              #승패 없음
              win_dct[team_2] = win_dct.get(team_2,0)
              lost_dct[team_2] = lost_dct.get(team_2,0)


            j=j+1
    
    group_results=[win_dct,draw_dct,lost_dct,team_wins_dct,goal_scored_dct,goal_against_dct]
    Group_standings[grp_name]=group_results


========Qualifying Games=========
[Group A Matches]
applied_score :  [2, 1] , predicted_score:  [1.883613109588623, 1.3126225471496582]
 Netherlands vs Senegal 
  Netherlands wins 

applied_score :  [2, 1] , predicted_score:  [2.129458427429199, 0.9105938673019409]
 Netherlands vs Ecuador 
  Netherlands wins 

applied_score :  [1, 2] , predicted_score:  [0.5354453921318054, 1.8808081150054932]
 Netherlands vs Qatar 
  Qatar wins 

applied_score :  [2, 1] , predicted_score:  [1.518472671508789, 1.0985525846481323]
 Senegal vs Ecuador 
  Senegal wins 

applied_score :  [1, 2] , predicted_score:  [0.8304263353347778, 1.899169921875]
 Senegal vs Qatar 
  Qatar wins 

applied_score :  [1, 2] , predicted_score:  [0.678215503692627, 1.5634466409683228]
 Ecuador vs Qatar 
  Qatar wins 

[Group B Matches]
applied_score :  [2, 1] , predicted_score:  [1.9493474960327148, 0.987572431564331]
 England vs United States 
  England wins 

applied_score :  [3, 1] , predicted_score:  [3.005863904953003, 